In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from extract.polocm import AP, Hypothesis, POLOCM
from traces import *
from convertor import TopoConvertor
from observation import PartialOrderedActionObservation, ObservedPartialOrderTraceList
from extract.model import *
import pulp as pl
from tabulate import tabulate

In [13]:
objects = {
    "A": PlanningObject("unknown", "A"),
    "B": PlanningObject("unknown", "B"),
    "C": PlanningObject("unknown", "C"),
}

actions = {
    "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
    "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
    "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
    "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
    "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
    "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
    "stackAB": Action("stack", [objects["A"], objects["B"]]),
    "stackBA": Action("stack", [objects["B"], objects["A"]]),
    "stackAC": Action("stack", [objects["A"], objects["C"]]),
    "stackCA": Action("stack", [objects["C"], objects["A"]]),
    "stackBC": Action("stack", [objects["B"], objects["C"]]),
    "stackCB": Action("stack", [objects["C"], objects["B"]]),
    "putdownA": Action("putdown", [objects["A"]]),
    "putdownB": Action("putdown", [objects["B"]]),
    "putdownC": Action("putdown", [objects["C"]]),
    "pickupA": Action("pickup", [objects["A"]]),
    "pickupB": Action("pickup", [objects["B"]]),
    "pickupC": Action("pickup", [objects["C"]]),
}

traces = TraceList(
    [
        Trace(
            [
                Step(State({}), actions["unstackAB"], 0),
                Step(State({}), actions["putdownA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["stackBC"], 3),
                Step(State({}), actions["unstackBC"], 4),
                Step(State({}), actions["stackBC"], 5),
                Step(State({}), actions["pickupA"], 6),
                Step(State({}), actions["stackAB"], 7),
            ]
        ),
        Trace(
            [
                Step(State({}),actions['unstackCB'], 0),
                Step(State({}),actions['putdownC'], 1),
                Step(State({}),actions['unstackBA'], 2),
                Step(State({}),actions['putdownB'], 3),
                Step(State({}),actions['pickupB'], 4),
                Step(State({}),actions['stackBC'], 5),
                Step(State({}),actions['pickupA'], 6),
                Step(State({}),actions['stackAB'], 7),

            ]
        ),
        Trace(
            [
                Step(State({}), actions["unstackCB"], 0),
                Step(State({}), actions["stackCA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["putdownB"], 3),
                Step(State({}), actions["pickupB"], 4),
                Step(State({}), actions["stackBC"], 5),
            ]
        ),
        
    ]
)


In [14]:
convertor = TopoConvertor("flex", True, True, 1)
po_tracelist, dod = traces.topo(convertor, 0.2)
obs_tracelist = po_tracelist.tokenize(PartialOrderedActionObservation, ObservedPartialOrderTraceList)

In [15]:
debug = {
    'pstep6': True,
    '2step0': True,
    'step7':True
}
model,AP, _ = POLOCM(obs_tracelist, debug=debug)

## Solution FO matrix

### Trace 0

#### Obj zero

╒════════════╤═════════════╤════════════╤══════════════╤════════════╤══════════════╤══════════════╤════════════╤═════════════╕
│            │   [2]pickup │   [5]stack │   [1]putdown │   [7]stack │   [0]unstack │   [4]unstack │   [3]stack │   [6]pickup │
╞════════════╪═════════════╪════════════╪══════════════╪════════════╪══════════════╪══════════════╪════════════╪═════════════╡
│ [2]pickup  │         nan │          0 │            0 │          0 │            0 │            0 │          1 │           0 │
├────────────┼─────────────┼────────────┼──────────────┼────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ [5]stack   │           0 │        nan │            0 │          0 │            0 │            0 │          0 │           1 │
├────────────┼─────────────┼────────────┼──────────────┼────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ [1]putdown │           1 │          0 │          nan │          0 │            0 │            0 │          0 

#### Obj B

...

╒═══════════╤════════════╤═════════════╤═════════════╤═════════════╤═══════════╤═══════════╕
│           │   pickup.1 │   putdown.1 │   unstack.2 │   unstack.1 │   stack.2 │   stack.1 │
╞═══════════╪════════════╪═════════════╪═════════════╪═════════════╪═══════════╪═══════════╡
│ pickup.1  │          0 │           1 │           0 │           0 │       nan │         1 │
├───────────┼────────────┼─────────────┼─────────────┼─────────────┼───────────┼───────────┤
│ putdown.1 │          1 │         nan │           0 │           0 │         1 │         0 │
├───────────┼────────────┼─────────────┼─────────────┼─────────────┼───────────┼───────────┤
│ unstack.2 │          1 │           0 │         nan │           1 │         1 │         0 │
├───────────┼────────────┼─────────────┼─────────────┼─────────────┼───────────┼───────────┤
│ unstack.1 │          0 │           1 │           0 │         nan │         0 │         1 │
├───────────┼────────────┼─────────────┼─────────────┼─────────────┼──

,unstack.0,putdown.0,pickup.0,stack.0
unstack.0,0,3,0,2
putdown.0,1,0,3,0
pickup.0,0,1,0,5
stack.0,1,0,3,0


Sort.1 AML:


,unstack.2,pickup.1,stack.1,unstack.1,stack.2,putdown.1
unstack.2,0,3,0,1,1,0
pickup.1,0,0,5,0,0,1
stack.1,0,0,0,1,3,0
unstack.1,0,0,2,0,0,3
stack.2,1,0,0,0,0,0
putdown.1,0,3,0,0,1,0


Zero Sort removed!
bindings:
{}

fluents:
{(F0S1state2 sort1),
 (F0S1state0 sort1),
 (F0S1state3 sort1),
 (F1S1state2 sort1),
 (F2S1state0 sort1),
 (F2S1state1 sort1),
 (F2S1state2 sort1),
 (F0S1state1 sort1),
 (F1S1state1 sort1),
 (F1S1state0 sort1)}

actions:
{(unstack sort1 sort1), (stack sort1 sort1), (pickup sort1), (putdown sort1)}



In [16]:
model.to_pddl('polocm')